Looking at Some ComCam DRP subtractions

Notebook by Michael Wood-Vasey: <wmwv@pitt.edu>  

In [ ]:
import matplotlib.pyplot as plt

from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay

import lsst.geom

import lsst.afw.image
from lsst.afw.math import Warper, WarperConfig
import lsst.afw.table

from lsst.ip.diffim import AlardLuptonSubtractConfig, AlardLuptonSubtractTask
from lsst.ip.diffim import GetTemplateConfig, GetTemplateTask
from lsst.ip.diffim import DetectAndMeasureConfig, DetectAndMeasureTask

In [ ]:
# Choose backend
afwDisplay.setDefaultBackend("firefly")
# afwDisplay.setDefaultBackend("matplotlib")

In [ ]:
# Load collection
repo = "embargo_new"
collection = "LSSTComCam/runs/DRP/20241101_20241113/w_2024_46/DM-47566"
instrument = "LSSTComCam"

butler = Butler(repo, collections=collection)

name_skymap = "lsst_cells_v1"
skymap = butler.get("skyMap", skymap=name_skymap, collections="skymaps")

### Identify data_ids from campaign

In [ ]:
band = "i"

datasetRefs_dia = butler.query_datasets("goodSeeingDiff_differenceExp", where=f"band='{band}'")

In [ ]:
verbose = False
if verbose:
    for dr in datasetRefs_dia:
        print(dr)

print(f"\nFound {len(datasetRefs_dia)} differenceExps")

In [ ]:
i = 50
data_id = datasetRefs_dia[i].dataId
print(data_id)

In [ ]:
dia = butler.get("goodSeeingDiff_differenceExp", dataId=data_id)
template = butler.get("goodSeeingDiff_templateExp", dataId=data_id)
calexp = butler.get("calexp", dataId=data_id)
src = butler.get("src", dataId=data_id)
dia_src = butler.get("goodSeeingDiff_diaSrc", dataId=data_id)
dia_src_table = butler.get("goodSeeingDiff_diaSrcTable", dataId=data_id)  # SDM-ified table.  Does not containg sky sources

In [ ]:
afw_display = afwDisplay.Display(frame=1)

afw_display.setMaskTransparency(80)
afw_display.scale("asinh", -2, 5)
afw_display.mtv(template)

In [ ]:
afw_display = afwDisplay.Display(frame=2)

afw_display.setMaskTransparency(80)
afw_display.scale("asinh", -2, 5)
afw_display.mtv(calexp)

In [ ]:
afw_display = afwDisplay.Display(frame=3)

afw_display.setMaskTransparency(100)
afw_display.scale("linear", "zscale")
afw_display.mtv(dia)

Then go to the Firefly window and click the "link" button in the toolbar (between the book of mask plans and the full-screen diagonal arrow).  Select "Align and Lock Options" -> "by Pixel at Image Centers" and then things will move together when you pan around and zoom in and out.  I don't know how to do this programatically as a set in the Notebook.

In [ ]:
snr_threshold = 7.5
max_science_snr = 200

# We're mixing columsn from dia_src and dia_src_table
# This is dangerous.  I don't think there is a gaurantee that the ordering is the same.
# But I want the flags from dia_src and the science flux from dia_src_table
good = ~dia_src["slot_Shape_flag"] & \
    (dia_src["base_PsfFlux_instFlux"] / dia_src["base_PsfFlux_instFluxErr"] > snr_threshold) & \
    ~dia_src["base_PixelFlags_flag_edge"] & \
    ((dia_src_table["scienceFlux"] / dia_src_table["scienceFluxErr"]) < max_science_snr) & \
    ~dia_src_table["pixelFlags_streak"]

# If I were doing just dia_src_table, I'd do something like this:
good = (dia_src_table["snr"] > snr_threshold) & \
    ~dia_src_table["shape_flag"] & \
    ~dia_src_table["pixelFlags_bad"] & \
    ~dia_src_table["pixelFlags_cr"] & \
    ((dia_src_table["scienceFlux"] / dia_src_table["scienceFluxErr"]) < max_science_snr)

good_dia_src = dia_src_table[good].copy(deep=True)


print(f"{len(good_dia_src)} good DIA sources found out of {len(dia_src)} detections.")

In [ ]:
for (x, y) in zip(good_dia_src["x"], good_dia_src["y"]):
    afw_display.dot("o", x, y, size=20, ctype="green")

In [ ]:
good_dia_src["scienceFlux"] / good_dia_src["scienceFluxErr"]

Here's a brief appendix to remind ourselves of the columns available in each of `dia_src_table` and `dia_src`:

In [ ]:
dia_src_table.columns

In [ ]:
dia_src.schema

In [ ]:
good_dia_src

In [ ]:
for c in good_dia_src.columns:
    print(good_dia_src[c])